Le but de ce notebook est d'extraire les résultats des différentes journées de football au sein du championnat de France (Ligue 1 & 2) et des autres grands championnat (DE, ESP, UK, IT).

Les données sont scraper sur le site de l'Equipe

In [1]:
from bs4 import BeautifulSoup
import urllib3
import requests
import pandas as pd
import re
import random
import time
import datetime
from copy import deepcopy
import numpy as np

In [ ]:
test_link = "https://www.lequipe.fr/Football/ligue-1/saison-2018-2019/page-calendrier-resultats/38e-journee"
token = "https://www.lequipe.fr/Football/ligue-1/saison-2018-2019/page-calendrier-resultats/{leg}-journee"

Ce que nous voulons:

- HomeTeam span
- AwayTeam 
- Score: class Teamscore div -> </a>

In [ ]:
#response = requests.get(test_link)
#soup = BeautifulSoup(response.text, 'html.parser')
#team_box = soup.find_all("span", {"class": "TeamScore__nameshort"})
#score_box = soup.find_all("div", {"class": "TeamScore__score TeamScore__score--ended"})

In [ ]:
bug_link = 'https://www.lequipe.fr/Football/ligue-1/saison-2007-2008/page-calendrier-resultats/24e-journee'

In [ ]:
response = requests.get(bug_link)
soup = BeautifulSoup(response.text, 'html.parser')
team_box = soup.find_all("span", {"class": "TeamScore__nameshort"})
score_box = soup.find_all("div", {"class": "TeamScore__score TeamScore__score--ended"})

In [ ]:
# soup

In [ ]:
conf = {
    'ligue-1' : {
        'country' : 'France',
        'first_season_start' : 2002,
        'last_season_start' : 2018,
        'nb_teams': 20,
        'url_template' : 'https://www.lequipe.fr/Football/ligue-1/saison-{season}/page-calendrier-resultats/{leg}-journee'
    },
    'ligue-2' : {
        'country' : 'France',
        'first_season_start' : 2002,
        'last_season_start' : 2018,
        'nb_teams': 20,
        'url_template' : 'https://www.lequipe.fr/Football/ligue-2/saison-{season}/page-calendrier-resultats/{leg}-journee'
    },
    
    'bundesliga' : {
        'country' : 'Germany',
        'first_season_start' : 2004,
        'last_season_start' : 2018,
        'nb_teams': 18,
        'url_template' : 'https://www.lequipe.fr/Football/championnat-d-allemagne/saison-{season}/page-calendrier-resultats/{leg}-journee'
    },
    
    'liga' : {
        'country' : 'Spain',
        'first_season_start' : 2004,
        'last_season_start' : 2018,
        'nb_teams': 20,
        'url_template' : 'https://www.lequipe.fr/Football/championnat-d-espagne/saison-{season}/page-calendrier-resultats/{leg}-journee'
    },
    
    'premier-league' : {
        'country' : 'England',
        'first_season_start' : 2004,
        'last_season_start' : 2018,
        'nb_teams': 20,
        'url_template' : 'https://www.lequipe.fr/Football/championnat-d-angleterre/saison-{season}/page-calendrier-resultats/{leg}-journee'
    },
    
    'serie-a' : {
        'country' : 'Italy',
        'first_season_start' : 2004,
        'last_season_start' : 2018,
        'nb_teams': 20,
        'url_template' : 'https://www.lequipe.fr/Football/championnat-d-italie/saison-{season}/page-calendrier-resultats/{leg}-journee'
    },
}

In [ ]:
conf_covid_season = {
    'ligue-1' : {
        'country' : 'France',
        'first_season_start' : 2019,
        'last_season_start' : 2019,
        'nb_teams': 20,
        'url_template' : 'https://www.lequipe.fr/Football/ligue-1/saison-{season}/page-calendrier-resultats/{leg}-journee'
    },
    'ligue-2' : {
        'country' : 'France',
        'first_season_start' : 2019,
        'last_season_start' : 2019,
        'nb_teams': 20,
        'url_template' : 'https://www.lequipe.fr/Football/ligue-2/saison-{season}/page-calendrier-resultats/{leg}-journee'
    },
    
    'bundesliga' : {
        'country' : 'Germany',
        'first_season_start' : 2019,
        'last_season_start' : 2019,
        'nb_teams': 18,
        'url_template' : 'https://www.lequipe.fr/Football/championnat-d-allemagne/saison-{season}/page-calendrier-resultats/{leg}-journee'
    },
    
    'liga' : {
        'country' : 'Spain',
        'first_season_start' : 2019,
        'last_season_start' : 2019,
        'nb_teams': 20,
        'url_template' : 'https://www.lequipe.fr/Football/championnat-d-espagne/saison-{season}/page-calendrier-resultats/{leg}-journee'
    },
    
    'premier-league' : {
        'country' : 'England',
        'first_season_start' : 2019,
        'last_season_start' : 2019,
        'nb_teams': 20,
        'url_template' : 'https://www.lequipe.fr/Football/championnat-d-angleterre/saison-{season}/page-calendrier-resultats/{leg}-journee'
    },
    
    'serie-a' : {
        'country' : 'Italy',
        'first_season_start' : 2019,
        'last_season_start' : 2019,
        'nb_teams': 20,
        'url_template' : 'https://www.lequipe.fr/Football/championnat-d-italie/saison-{season}/page-calendrier-resultats/{leg}-journee'
    },
}

In [ ]:
conf_redo = {
    'ligue-1' : {
        'country' : 'France',
        'first_season_start' : 2019,
        'last_season_start' : 2019,
        'nb_teams': 20,
        'url_template' : 'https://www.lequipe.fr/Football/ligue-1/saison-{season}/page-calendrier-resultats/{leg}-journee'
    }
}

In [22]:
re.findall(r'([0-9îê\'üöëéèâÉa-zA-Z-\s?]+)', "L'dfd")

["L'dfd"]

In [30]:
re.findall(r'>([0-9]+)</span', ">9</span")

[]

In [71]:
def pause():
    """
    The goal of this section is to pause the execution before 2 scraping executions
    :return: wait 1, 2, 3, 4 or 4 seconds
    """
    time_break = random.randint(1, 4)
    return time.sleep(time_break)

def get_url_leg(url_template, season, nbr, pattern_first=None):
    """
    :param url_template: str: template of the url to be used
    :param season: str
    :param nbr: int: leg number
    
    :return a completed url 
    """
    pattern_first = "1ere" if pattern_first is None else pattern_first
    leg_token = "{}e".format(nbr) if nbr > 1 else pattern_first
    return url_template.format(leg=leg_token, season=season)


def transform_box_to_data(box, regex_pattern, data_name, data_type):
    """
    :param box: eautifulSoup box corresponding to 1 part of the data
    :param regex_pattern: regexp value extract the relevant data
    :param data_name: str: name of the data we're interested in
    :param data_type: str: converting type for the extracted data
    
    :return dataframe with all data retrieved from the box
    """
    elements = []
    # extract from box
    for box_elem in box:
        for x in box_elem:
            elem = re.findall(regex_pattern, str(x))
            if elem:
                # print(elem)
                elem = int(elem[0]) if data_type == 'int' else elem[0]
                elements.append(elem)
    
    # build data
    data = {'key': [('Home' if i%2==0 else 'Away') + '_' + str(i//2) for i, _ in enumerate(elements)],
           data_name: elements}
    return pd.DataFrame(data=data)



In [31]:
url = 'https://www.lequipe.fr/Football/championnat-d-espagne/saison-2015-2016/page-calendrier-resultats/16e-journee'

In [56]:
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

team_box = soup.find_all("span", {"class": "TeamScore__nameshort"})
score_box = soup.find_all("div", {"class": ["TeamScore__score TeamScore__score--ended", 
                                            "TeamScore__score TeamScore__score--ended TeamScore__score--long"
                                            ]})
# "TeamScore__score TeamScore__score--ended TeamScore__score--long"

goal_scored_df = transform_box_to_data(box=score_box, regex_pattern=r'>([0-9]+)</span',  
                                       data_name='goals_scored', data_type='int')

['2']
['2']
['1']
['0']
['0']
['0']
['2']
['0']
['10']
['2']
['0']
['2']
['0']
['2']
['2']
['0']
['1']
['0']
['1']
['3']


In [57]:
goal_scored_df

,key,goals_scored
0,Home_0,2
1,Away_0,2
2,Home_1,1
3,Away_1,0
4,Home_2,0
5,Away_2,0
6,Home_3,2
7,Away_3,0
8,Home_4,10
9,Away_4,2


In [60]:
def get_leg_data(season:str, leg:int, championship: str, country: str, url: str):
    """
    :param season: str: 
    :param leg: int: leg number
    :param championship: str: name of the championship
    :param country: str: name of the country
    :param url: str: url from which data is retrieved
    
    :return dataframe with all retrieved data 
    """
    
    # print('leg : {}'.format(leg))
    
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')

    team_box = soup.find_all("span", {"class": "TeamScore__nameshort"})
    score_box = soup.find_all("div", {"class": ["TeamScore__score TeamScore__score--ended", 
                                            "TeamScore__score TeamScore__score--ended TeamScore__score--long"
                                            ]})
    
    goal_scored_df = transform_box_to_data(box=score_box, regex_pattern=r'>([0-9]+)</span',  
                                           data_name='goals_scored', data_type='int')
    team_df = transform_box_to_data(box=team_box, regex_pattern=r'>([0-9îê\'üöëéèâÉa-zA-Z-\s?]+)</span', 
                                    data_name='team', data_type='str')
    
    leg_df = pd.merge(team_df, goal_scored_df, on='key')
    leg_df['leg'] = leg
    leg_df['season'] = season
    leg_df['championship'] = championship
    leg_df['country'] = country
    
    leg_df[['play', 'game_number']] = leg_df[['key']].apply(lambda k: k['key'].split('_'), 
                                                            axis=1, 
                                                            result_type='expand')

    leg_df['opponent_key'] = leg_df['key'].apply(
        lambda k: '{}'.format('Away' if k.startswith('Home') else 'Home') + '_' + k.split('_')[-1]
    )


    opponent_df = leg_df[['key', 'team', 'goals_scored']]
    opponent_df.rename(columns={'key': 'opponent_key', 'team': 'opponent', 'goals_scored': 'goals_conceded'},
                      inplace=True)

    leg_df = leg_df.merge(opponent_df, on='opponent_key')

    leg_df['nb_points'] = leg_df[['goals_scored', 'goals_conceded']].apply(
        lambda r : 3*(r['goals_scored']>r['goals_conceded'])+ (r['goals_scored']==r['goals_conceded']),
        axis=1
    )
        
    return leg_df[['country', 'season', 'leg', 'team', 'play', 'goals_scored', 'opponent', 
                   'goals_conceded', 'nb_points']]

In [61]:
def get_leg_data_covid_missing(leg:int, championship: str, url: str):
    """
    :param season: str: 
    :param leg: int: leg number
    :param championship: str: name of the championship
    :param country: str: name of the country
    :param url: str: url from which data is retrieved
    
    :return dataframe with all retrieved data 
    """
    
    # print('leg : {}'.format(leg))
    
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')

    team_box = soup.find_all("span", {"class": "TeamScore__nameshort"})
    
    team_df = transform_box_to_data(box=team_box, regex_pattern=r'>([0-9îê\'üöëéèâÉa-zA-Z-\s?]+)</span', 
                                    data_name='team', data_type='str')
    
    leg_df = deepcopy(team_df)
    leg_df['leg'] = leg
    leg_df['season'] = '2019-2020'
    leg_df['championship'] = championship
    leg_df['country'] = 'France'

    
    leg_df[['play', 'game_number']] = leg_df[['key']].apply(lambda k: k['key'].split('_'), 
                                                            axis=1, 
                                                            result_type='expand')

    leg_df['opponent_key'] = leg_df['key'].apply(
        lambda k: '{}'.format('Away' if k.startswith('Home') else 'Home') + '_' + k.split('_')[-1]
    )


    opponent_df = leg_df[['key', 'team']]
    opponent_df.rename(columns={'key': 'opponent_key', 'team': 'opponent'},
                      inplace=True)

    leg_df = leg_df.merge(opponent_df, on='opponent_key')

    
        
    return leg_df[['country', 'season', 'leg', 'team', 'play', 'opponent']]

In [62]:
def get_season_data(url_template, championship, country, season, nb_legs, pattern_first=None):
    """
    :param url_template: str: template to use in order to get data for a specific championship and season 
    :param championship: str: name of the championship
    :param country: str: name of the country
    :param season: str
    :param nb_legs: int: number of legs to be played for a season
    
    :return dataframe containing all data for a specific championship and season 
    """
    leg_dfs_ll = []
    for leg in range(1, nb_legs+1):   
        url = get_url_leg(url_template=url_template, season=season, nbr=leg, pattern_first=pattern_first)
        try:
            leg_df = get_leg_data(season=season, leg=leg, championship=championship,
                                  country=country, url=url)
            leg_dfs_ll.append(leg_df)
        except ValueError:
            
            # covid exception
            if (country=='France' and season=='2019-2020' and leg > 26) :
                try:
                    leg_df = get_leg_data_covid_missing(leg=leg, championship=championship, url=url)
                    leg_dfs_ll.append(leg_df)
                except ValueError:
                    print("""Could not get data : \n
                                championship : {champ},\n
                                season : '2019-2020', \n
                                leg : {leg}""".format(champ=championship, leg=leg))
            else:
                print("""Could not get data : \n
                championship : {champ},\n
                season : {season}, \n
                leg : {leg}""".format(champ=championship, season=season, leg=leg))
        pause()
        
    return pd.concat(leg_dfs_ll, ignore_index=True) 

In [63]:
def get_championship_data(championship, c_conf, pattern_first=None):
    """
    :param championship: str: name of the championship
    :param c_conf: dict: configuration file focused on the specified championship
    
    :return dataframe with all the data for the corresponding championship
    """
    start = c_conf['first_season_start']
    end = c_conf['last_season_start']
    nb_teams = c_conf['nb_teams']
    nb_legs = 2*(nb_teams-1)
    country = c_conf['country']
    url_template = c_conf['url_template']
    season_dfs_ll = []
    for year in range(start, end+1):
        season = '{s}-{e}'.format(s=year, e=year+1)
        print('{}, season {}'.format(championship, season))
        season_df = get_season_data(url_template=url_template,
                                    championship=championship,
                                    country=country,
                                    season=season,
                                    nb_legs=nb_legs,
                                    pattern_first=pattern_first)
        season_dfs_ll.append(season_df)
        pause()
        
    return pd.concat(season_dfs_ll, ignore_index=True)

In [64]:
def get_data(conf, pattern_first=None):
    """
    :param conf: dict: configuration file containing all needed informations to collect the data
    
    :return None
    """
    for championship, c_conf in conf.items():
        data = get_championship_data(championship=championship, c_conf=c_conf, pattern_first=pattern_first)
        name = championship+'_data_{start}_{end}'.format(start=c_conf['first_season_start'],
                                                        end=c_conf['last_season_start']+1)
        data.to_csv(name)

In [ ]:
%%time
get_data(conf=conf_redo, pattern_first = "1re")

In [ ]:
df =pd.read_csv('ligue-1_data_2019_2020')

In [ ]:
df[['leg', 'season']].groupby(by=['leg']).count()

In [ ]:
df[df.leg==28]

In [8]:
def get_missing_leg(url_template, season, leg, championship, country, pattern_first=None ):
    url = get_url_leg(url_template=url_template, season=season, nbr=leg, pattern_first=pattern_first)
    print(url)
    try:
        leg_df = get_leg_data(season=season, leg=leg, championship=championship,
                              country=country, url=url)
        csv_file = f'missing_{championship}_{season}_{leg}'
        
        print(csv_file)
        leg_df.to_csv(csv_file)
        return leg_df
    except ValueError:
        print("""Could not get data : \n
        championship : {champ},\n
        season : {season}, \n
        leg : {leg}""".format(champ=championship, season=season, leg=leg))

In [58]:
url_template = 'https://www.lequipe.fr/Football/championnat-d-espagne/saison-{season}/page-calendrier-resultats/{leg}-journee'
season = '2015-2016'
leg = 16
championship = 'liga'
country = 'Spain'
pattern_first = "1re"

In [65]:
df = get_missing_leg(url_template=url_template, 
                    season=season,
                    leg=leg, 
                    championship=championship,
                    country=country ,
                    pattern_first=pattern_first)

https://www.lequipe.fr/Football/championnat-d-espagne/saison-2015-2016/page-calendrier-resultats/16e-journee
['2']
['2']
['1']
['0']
['0']
['0']
['2']
['0']
['10']
['2']
['0']
['2']
['0']
['2']
['2']
['0']
['1']
['0']
['1']
['3']
['Valence CF']
['Getafe']
['Espanyol Barcelone']
['Las Palmas']
['Betis Séville']
['Séville FC']
['La Corogne']
['Eibar']
['Real Madrid']
['Rayo Vallecano']
['Grenade CF']
['Celta Vigo']
['Real Sociedad']
['Villarreal']
['Athletic Bilbao']
['Levante']
['Malaga']
['Atlético de Madrid']
['Gijon']
['FC Barcelone']
missing_liga_2015-2016_16


In [13]:
def add_missing_dataframe(stem_dataframe_csv_path, 
                          missing_dataframe_csv_path,
                         sort_by_value=None):
    
    _, _, season, leg = missing_dataframe_csv_path.split("_")
    leg=int(leg)
    
    stem = pd.read_csv(stem_dataframe_csv_path).drop(columns='Unnamed: 0')
    
    stem = deepcopy(stem[(stem.season!=season) | (stem.leg!=leg)])
    
    missing = pd.read_csv(missing_dataframe_csv_path).drop(columns='Unnamed: 0')
    
    glued = pd.concat([stem, missing], ignore_index=True).drop_duplicates()
    
    if sort_by_value:
        dg = glued.sort_values(by=sort_by_value, axis=0, ascending=True).reset_index(drop=True)
    
    dg.to_csv(stem_dataframe_csv_path)

In [ ]:
def add_record_to_dataframe(stem_dataframe_csv_path, 
                          record_df,
                         sort_by_value=None):
    stem = pd.read_csv(stem_dataframe_csv_path).drop(columns='Unnamed: 0')
    
    glued = pd.concat([stem, record_df], ignore_index=True).drop_duplicates()
    
    if sort_by_value:
        dg = glued.sort_values(by=sort_by_value, axis=0, ascending=True).reset_index(drop=True)
    
    dg.to_csv(stem_dataframe_csv_path)

In [ ]:
"s_s_d_sdsd".split("_")

In [21]:
# Bundesliga
add_missing_dataframe(stem_dataframe_csv_path='bundesliga_data_2004_2019',
                      missing_dataframe_csv_path='missing_bundesliga_2011-2012_1',
                      sort_by_value=['season', 'leg']
                     )

In [17]:
# Bundesliga
add_missing_dataframe(stem_dataframe_csv_path='bundesliga_data_2004_2019',
                      missing_dataframe_csv_path='missing_bundesliga_2014-2015_1',
                      sort_by_value=['season', 'leg']
                     )

In [ ]:
# Ligue 1
add_missing_dataframe(stem_dataframe_csv_path='ligue-1_data_2002_2019',
                      missing_dataframe_csv_path='missing_ligue-1_2007-2008_24',
                      sort_by_value=['season', 'leg']
                     )

In [ ]:
# Ligue 1
add_missing_dataframe(stem_dataframe_csv_path='ligue-1_data_2002_2019',
                      missing_dataframe_csv_path='missing_ligue-1_2015-2016_15',
                      sort_by_value=['season', 'leg']
                     )

In [ ]:
# Ligue 2
add_missing_dataframe(stem_dataframe_csv_path='ligue-2_data_2002_2019',
                      missing_dataframe_csv_path='missing_ligue-2_2015-2016_6',
                      sort_by_value=['season', 'leg']
                     )

In [ ]:
# Serie A
add_missing_dataframe(stem_dataframe_csv_path='serie-a_data_2004_2019',
                      missing_dataframe_csv_path='missing_serie-a_2010-2011_17',
                      sort_by_value=['season', 'leg']
                     )

In [ ]:
# Liga
add_missing_dataframe(stem_dataframe_csv_path='liga_data_2004_2019',
                      missing_dataframe_csv_path='missing_liga_2011-2012_27',
                      sort_by_value=['season', 'leg']
                     )

In [72]:
# Liga
add_missing_dataframe(stem_dataframe_csv_path='liga_data_2004_2019',
                      missing_dataframe_csv_path='missing_liga_2015-2016_16',
                      sort_by_value=['season', 'leg']
                     )

In [ ]:
# Serie A (2nd time)
add_missing_dataframe(stem_dataframe_csv_path='serie-a_data_2004_2019',
                      missing_dataframe_csv_path='missing_serie-a_2004-2005_1',
                      sort_by_value=['season', 'leg']
                     )

In [ ]:
add_missing_dataframe(stem_dataframe_csv_path='serie-a_data_2004_2019',
                      missing_dataframe_csv_path='missing_serie-a_2004-2005_34',
                      sort_by_value=['season', 'leg']
                     )

In [ ]:
record = pd.DataFrame(data={'country' : ['France']*2,
'season': ['2019-2020']*2,
'leg': [28]*2,
'team':['Strasbourg', 'Paris-SG'],
'play':['Home', 'Away'],
'goals_scored':[np.nan, np.nan],
'opponent':['Paris-SG', 'Strasbourg'],
'goals_conceded':[np.nan,np.nan],
'nb_points':[np.nan,np.nan]})

In [ ]:
record

In [ ]:
add_record_to_dataframe(stem_dataframe_csv_path='ligue-1_data_2019_2020', 
                          record_df=record,
                         sort_by_value=['season', 'leg'])

In [ ]:
record = pd.DataFrame(data={'country' : ['France']*2,
'season': ['2011-2012']*2,
'leg': [19]*2,
'team':['Saint-Étienne', 'Paris-SG'],
'play':['Home', 'Away'],
'goals_scored':[0, 1],
'opponent':['Paris-SG', 'Saint-Étienne'],
'goals_conceded':[1,0],
'nb_points':[0,3]})

In [ ]:
add_record_to_dataframe(stem_dataframe_csv_path='ligue-1_data_2002_2019', 
                          record_df=record,
                         sort_by_value=['season', 'leg'])

In [ ]:
record = pd.DataFrame(data={'country' : ['France']*2,
'season': ['2014-2015']*2,
'leg': [1]*2,
'team':['Monaco', 'Lorient'],
'play':['Home', 'Away'],
'goals_scored':[1, 2],
'opponent':['Lorient', 'Monaco'],
'goals_conceded':[2,1],
'nb_points':[0,3]})

In [ ]:
add_record_to_dataframe(stem_dataframe_csv_path='ligue-1_data_2002_2019', 
                          record_df=record,
                         sort_by_value=['season', 'leg'])